# Práctica 3 - Multiclasificadores y selección de variables

**Nombres:** 
 - **Alejandro Moya Moya** - LinkedIn: https://www.linkedin.com/in/alejandro-moya-moya/
 - **Jorge Valero Molina** - LinkedIn: https://www.linkedin.com/in/jorge-valero-molina-0a2a0512b/
 
**Año:** 2017/18

**Revisión:** 0.8

# Indice:
* [1. Antes de empezar...](#1.-Antes-de-empezar...)
* [2. Carga de los Datasets](#2.-Carga-de-los-Datasets)
* [3. Uso y ajuste de métodos basados en ensembles](#3.-Uso-y-ajuste-de-métodos-basados-en-ensembles)
    * [3.1 Implementación básica de un ensemble de manera manual](#3.1-Implementación-básica-de-un-ensemble-de-manera-manual)
    * [3.2 Uso de Bagging, Boosting, Random Forest y Gradient Boosting](#3.2-Uso-de-Bagging,-Boosting,-Random-Forest-y-Gradient-Boosting)
        * [3.2.1 Bagging](#3.2.1-Bagging)
            * [3.2.1.1 Bagging del dataset Pima](#3.2.1.1-Bagging-del-dataset-Pima)
            * [3.2.1.2 Bagging del dataset Wisconsin](#3.2.1.2-Bagging-del-dataset-Wisconsin)
        * [3.2.2 Boosting](#3.2.2-Boosting)
            * [3.2.2.1 Boosting del dataset Pima](#3.2.2.1-Boosting-del-dataset-Pima)
            * [3.2.2.2 Boosting del dataset Wisconsin](#3.2.2.2-Boosting-del-dataset-Wisconsin)
        * [3.2.3 Random Forest](#3.2.3-Random-Forest)
            * [3.2.3.1 Random Forest del dataset Pima](#3.2.3.1-Random-Forest-del-dataset-Pima)
            * [3.2.3.2 Random Forest del dataset Wisconsin](#3.2.3.2-Random-Forest-del-dataset-Wisconsin)
        * [3.2.4 Gradient Boosting](#3.2.4-Gradient-Boosting)
            * [3.2.4.1 Gradient Boosting del dataset Pima](#3.2.4.1-Gradient-Boosting-del-dataset-Pima)
            * [3.2.4.2 Gradient Boosting del dataset Wisconsin](#3.2.4.2-Gradient-Boosting-del-dataset-Wisconsin)
    * [3.3 Resultados obtenidos tras usar Bagging, Boosting, Random Forest, Gradient Boosting y nuestro Ensemble](#3.3-Resultados-obtenidos-tras-usar-Bagging,-Boosting,-Random-Forest,-Gradient-Boosting-y-nuestro-Ensemble)
* [4. Selección de variables](#4.-Selección-de-variables)
    * [4.1 Método filter basado en rankings y la importancia de las variables para evaluar distintos subconjuntos](#4.1-Método-filter-basado-en-rankings-y-la-importancia-de-las-variables-para-evaluar-distintos-subconjuntos)
        * [4.1.1 Método filter basado en rankings](#4.1.1-Método-filter-basado-en-rankings)
            * [4.1.1.1 Método filter basado en rankings con dataset Pima](#4.1.1.1-Método-filter-basado-en-rankings-con-dataset-Pima)
            * [4.1.1.2 Método filter basado en rankings con dataset Wisconsin](#4.1.1.2-Método-filter-basado-en-rankings-con-dataset-Wisconsin)
        * [4.1.2 Método filter basado en la importancia de las variables](#4.1.2-Método-filter-basado-en-la-importancia-de-las-variables)
            * [4.1.2.1 Método filter basado en la importancia de las variables con dataset Pima](#4.1.2.1-Método-filter-basado-en-la-importancia-de-las-variables-con-dataset-Pima)
            * [4.1.2.2 Método filter basado en la importancia de las variables con dataset Wisconsin](#4.1.2.2-Método-filter-basado-en-la-importancia-de-las-variables-con-dataset-Wisconsin)
    * [4.2 Algoritmo de búsqueda recursiva wrapper](#4.2-Algoritmo-de-búsqueda-recursiva-wrapper)

## 1. Antes de empezar...

In [1]:
# Cargamos las librerias necesarias para poder realizar la práctica.

import numpy as np
import pandas as pd
import sklearn.metrics as metrics
from scipy import stats, integrate
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.utils import resample
from sklearn import neighbors
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import SelectFromModel

import seaborn as sns
sns.set(color_codes=True)

# Este codigo configura matplotlib para una representacion adecuada
%matplotlib inline
mpl.rcParams["figure.figsize"] = "8, 4"
import warnings
warnings.simplefilter("ignore")

# Este codigo hace una tabla HTML
from IPython.display import display, HTML

def printTable(list):
    table = """<table>%s</table>"""
    row = """<tr>%s</tr>"""
    cell = """<td>%s</td>"""
    report =  table % ''.join([row % (cell % x[0] + cell % x[1]) for x in results])
    display(HTML(report))

# Establecemos una semilla para poder obtener siempre los resultados
import random
seed=1234
np.random.seed(seed)

# 2. Carga de los Datasets
Cargamos un Dataset, para esta práctica usaremos 'Pima' y 'Wisconsin'

In [2]:
# Cargamos el fichero con el conjunto de datos. OJO: colocar el fichero de datos en el mismo directorio que esta libreta
df = {}
dfAttributes = {}
dfLabel = {}
df[1] = pd.read_csv("pima.csv", dtype={ "label": 'category'})

# Preprocesamiento de pima manual (Cambiar los 0's por la media)
df[1]["plas"].replace(0, df[1]["plas"].mean(), inplace=True)
df[1]["pres"].replace(0, df[1]["pres"].mean(), inplace=True)
df[1]["skin"].replace(0, df[1]["skin"].mean(), inplace=True)
df[1]["insu"].replace(0, df[1]["insu"].mean(), inplace=True)
df[1]["mass"].replace(0, df[1]["mass"].mean(), inplace=True)

dfAttributes[1] = df[1].drop('label', 1)
dfLabel[1] = df[1]['label']

df[2] = pd.read_csv("wisconsin.csv", dtype={ "label": 'category'})

# Preprocesamiento de Wisconsin manual (Eliminamos la columna patientId y cambiamos los nan's por la media)
df[2].drop('patientId', 1, inplace=True)

df[2]["clumpThickness"].replace(np.nan, df[2]["clumpThickness"].mean(), inplace=True)
df[2]["cellSize"].replace(np.nan, df[2]["cellSize"].mean(), inplace=True)
df[2]["CellShape"].replace(np.nan, df[2]["CellShape"].mean(), inplace=True)
df[2]["marginalAdhesion"].replace(np.nan, df[2]["marginalAdhesion"].mean(), inplace=True)
df[2]["epithelialSize"].replace(np.nan, df[2]["epithelialSize"].mean(), inplace=True)
df[2]["bareNuclei"].replace(np.nan, df[2]["bareNuclei"].mean(), inplace=True)
df[2]["blandChromatin"].replace(np.nan, df[2]["blandChromatin"].mean(), inplace=True)
df[2]["normalNucleoli"].replace(np.nan, df[2]["normalNucleoli"].mean(), inplace=True)
df[2]["mitoses"].replace(np.nan, df[2]["mitoses"].mean(), inplace=True)

dfAttributes[2] = df[2].drop('label', 1)
dfLabel[2] = df[2]['label']


In [3]:
# Dividimos cada Dataset en train/test para realizar los experimentos.
# Para ello realizados un Holdout estratificado.
from sklearn.model_selection import train_test_split
train_atts = {}
test_atts = {}
train_label = {}
test_label = {}
# df[1]
train_atts[1], test_atts[1], train_label[1], test_label[1] = train_test_split( 
    dfAttributes[1],
    dfLabel[1],
    test_size=0.2,
    random_state=seed,
    stratify=dfLabel[1])

# df[2]
train_atts[2], test_atts[2], train_label[2], test_label[2] = train_test_split( 
    dfAttributes[2],
    dfLabel[2],
    test_size=0.2,
    random_state=seed,
    stratify=dfLabel[2])


# 3. Uso y ajuste de métodos basados en ensembles

## 3.1 Implementación básica de un ensemble de manera manual

In [4]:
# Ensemble básico
# Parametros:
    # train_atts [Dict]: conjunto de entrenamiento que contiene los atributos (solo se usa para aprender un modelo)
    # train_label [Dict]: conjunto de entrenamiento que contiene las soluciones (solo se usa para aprender un modelo)
    # test_atts [Dict]: conjunto de test que contiene los atributos (solo se usar para validar)
    # test_label [Dict]: conjunto de test que contiene los soluciones (solo se usa para validar)
    # n_estimator [Integer]: número de clasificadores a emplear
    # reemplazo [True,False]: indica si el muestreo de las instancias se realizará con/sin remplazo
    # n_sample [Intenger]: número de instancias a usar para cada clasificador
    # random [Intenger]: semilla
    # kn [Intenger]: número de vecinos a usar para cada clasificador
    # muestrAtts [Float - [0.0,1.0]]: indica si se hace muestreo o no de atributos. 1.0: sin muestreo, <1.0: muestreo
    #                                 más o menos agresivo.
def ensemble(train_atts, train_label, test_atts, test_label, n_estimator, reemplazo, n_sampl, random, kn, muestrAtts):
    # Definimos una lista que contendrá el número de clasificadores a usar definido por el usuario
    knn = [None] * n_estimator
    # Lista auxiliar, al comienzo se usa para guardar el conjunto de train_label de las instancias muestreadas
    lista = [None] * n_sampl
    # Lista que contendrá el resultado de la prediccion sobre test
    result = [None] * len(test_atts)
    # Por cada estimador indicado por el usuario...
    for i in range(0,n_estimator):
        # Realizamos una copia de test y training (esto solo tiene utilidad cuando hay muestreo de variables)
        train_attsCopy = train_atts.copy()
        test_attsCopy = test_atts.copy()
        # Comprobamos si hay muestreo de variables
        if muestrAtts != 1:
            # Si la hay...
            auxRandom = 0
            # Por cada varible predictora...
            for x in train_atts.keys():
                # Obtenemos un aleatorio entre 0.0 y 0.999999 y comprobamos si eliminamos o no la variable
                np.random.seed(random+auxRandom)
                if (np.random.random())>=muestrAtts:
                    del train_attsCopy[x]
                    del test_attsCopy[x]
                auxRandom+=1
                # Si se da el caso de que se eliminan más de la mitad de las variables, se dejará de seguir eliminando
                # con esto evitamos eliminar todas las variables y que el algoritmo arroje un error.
                if len(train_attsCopy.keys())<=(len(train_atts.keys())/2):
                    break
        # Obtenemos una lista de datos muestreada (con o sin remplazo, según indique "reemplazo")
        listaDatosMuestreados = {}
        listaDatosMuestreados = resample(train_attsCopy, n_samples=n_sampl, random_state=random, replace=reemplazo)
        # Obtenemos los indices de todas las instancias con el fin de poder localizarlas en train_label.
        indices = listaDatosMuestreados.index
        for k in range(0,n_sampl):
            lista[k]=(train_label.get(indices[k]))
        # Aprendemos un modelo con los vecinos indicados por el usuario
        model = neighbors.KNeighborsClassifier(kn)
        # Lo entrenamos con los datos de train muestreados
        knn[i] = model.fit(listaDatosMuestreados, lista)
        # Nos guardamos la prediccion realizada sobre el conjunto de test
        knn[i]=knn[i].predict(test_attsCopy)
        # Cambiamos el random con el fin de que los nuevos estimadores no sean todos iguales
        random+=25
    # Restablecemos el random impuesto al comienzo de la práctica, con el fin de evitar cambios no deseados
    np.random.seed(seed)
    
    # Ahora realizamos un agrupamiento de predicciones para cada instancia de test, para ello escogemos la predicci
    for j in range(0, len(test_atts)):
        # Restablecemos la lista auxiliar y ahora tendrá la función de almacenar todas las predicciones realizadas por
        # cada uno de los estimadores sobre una determinada instancia del conjunto de test
        lista = [None] * n_estimator
        # "Rescatamos" la prediccion realizada por cada estimador
        for i in range(0,n_estimator):
            lista.append(knn[i][j])
    
        mayor = 0
        clase = None
        # Una vez obtenida todas las predicciones sobre una instancia de test sobre una lista, procedemos a realizar
        # un voto por la mayoria, para ello comprobaremos cada una de las categorias que puede tomar la variable clase
        for i in range(0, len(test_label.cat.categories)):
            # Realizamos el recuento de número de apariciones de una categoria y lo almacenamos...
            aux = lista.count(test_label.cat.categories[i])
            # Si es mayor a lo que teníamos almacenado, nos lo guardamos
            if aux >= mayor:
                mayor = aux
                clase = test_label.cat.categories[i]
        # Una vez comprobada todas las posibles categorias que puede tomar la variable clase, almacenamos la prediccion
        # mayoritaria en result
        result[j] = clase
    # Por último, retornamos una lista con todas las predicciones realizadas sobre el conjunto de test
    return result

Vamos a probar nuestro ensemble con el dataset Pima con los siguiente parámetros:
    - train_atts [Dict]: train_atts[1]
    - train_label [Dict]: train_label[1]
    - test_atts [Dict]: test_atts[1]
    - test_label [Dict]: test_label[1]
    - n_estimator [Integer]: 100 clasificadores a emplear
    - reemplazo [True,False]: True, muestreo de las instancias con remplazo
    - n_sample [Intenger]: 150 instancias para cada clasificador
    - random [Intenger]: seed, semilla establecida en el sistema
    - kn [Intenger]: 5 vecinos para cada clasificador
    - muestrAtts [Float - [0.0,1.0]]: 0.5, con muestreo de atributos al 50%

In [5]:
clasif=ensemble(train_atts[1],train_label[1], test_atts[1], test_label[1], 100,True,150,seed,5,0.5)
# Validamos las predicciones obtenidas a partir de test por nuestro ensemble con respecto a su categoria correcta,
# obteniendo así la bondad de nuestro ensemble ante nuevos casos.
metrics.accuracy_score(test_label[1],clasif)

0.70779220779220775

Ahora, procedemos a hacer lo mismo con Wisconsin, cambiando los parámetros de train_atts, train_label, test_atts y test_label con los correspondientes a Wisconsin.

In [6]:
clasif=ensemble(train_atts[2],train_label[2], test_atts[2], test_label[2], 100,True,150,seed,5,0.5)
metrics.accuracy_score(test_label[2],clasif)

0.94999999999999996

## 3.2 Uso de Bagging, Boosting, Random Forest y Gradient Boosting

## 3.2.1 Bagging

In [7]:
# Funcion que itera los diferentes parámetros que puede tomar un ensemble bagging, para ello itera los parámetros
# pasados por parámetros, desde lo mínimo hasta lo indicado por el usuario
def baggingMax(train_atts, train_label, cv, scoring,  estimator, n_estimators, max_samples, random_state):
    # Nos almacenamos el Score y la configuración que ha obtenido el mejor Score para mostrarlo en pantalla
    maxScore = 0.0
    config = (None, None, None)
    for i in range(1,n_estimators+1):
        j=0.1
        while j<=max_samples:
            for k in [True,False]:
                # Se ejecuta el ensemble...
                bagg = BaggingClassifier(estimator, 
                          n_estimators = i,
                          max_samples = j,
                          bootstrap = k,
                          random_state=random_state)
                # Y se hace una validación cruzada con train
                scores_bagg = cross_val_score(bagg, train_atts, train_label, cv=cv, scoring=scoring)
                # Si el score ha sido mejor que el que teníamos, lo almacenamos y lo mostramos en pantalla como
                # posible candidato
                if scores_bagg.mean() > maxScore:
                    maxScore = scores_bagg.mean()
                    config = (i,j,k)
                    print("Configuracion candidata: ")
                    print(scoring, ": %0.2f (+/- %0.2f)" % (scores_bagg.mean(), scores_bagg.std()*2))
                    print("Estimator: " , estimator)
                    print("N_estimators: " , config[0])
                    print("Max_Samples: " , config[1])
                    print("Bootstrap: " , config[2])
                    print("Random_state: " , random_state)
                    
            j+=0.1
    # Configuración que ha obtenido el mejor score
    print("---------------- RESULTADO --------------")
    print("Configuracion mas optima: ")
    print(scoring , ": " , maxScore)
    print("Estimator: " , estimator)
    print("N_estimators: " , config[0])
    print("Max_Samples: " , config[1])
    print("Bootstrap: " , config[2])
    print("Random_state: " , random_state)

__Nota__: dado que todos los algoritmos creados para obtener la mejor configuración son similares (solo cambian los parámetros a iterar y el ensemble a ejecutar), solo se comentará este algoritmo, los demás se dan por explicados.

### 3.2.1.1 Bagging del dataset Pima

In [8]:
baggingMax(train_atts[1], train_label[1], 3, "accuracy", tree.DecisionTreeClassifier(), 60, 1, seed)

Configuracion candidata: 
accuracy : 0.60 (+/- 0.10)
Estimator:  DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
N_estimators:  1
Max_Samples:  0.1
Bootstrap:  True
Random_state:  1234
Configuracion candidata: 
accuracy : 0.69 (+/- 0.05)
Estimator:  DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
N_estimators:  1
Max_Samples:  0.1
Bootstrap:  False
Random_state:  1234
Configuracion candidata: 

Configuracion candidata: 
accuracy : 0.79 (+/- 0.04)
Estimator:  DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
N_estimators:  21
Max_Samples:  0.2
Bootstrap:  True
Random_state:  1234
Configuracion candidata: 
accuracy : 0.79 (+/- 0.04)
Estimator:  DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
N_estimators:  22
Max_Samples:  0.2
Bootstrap:  True
Random_state:  1234
Configuracion candidata:

Cogemos los parámetros que han maximizado el score del ensemble y lo ejecutamos a parte con el fin de obtener su Accuracy con respecto al conjunto de test:

In [9]:
bagg = BaggingClassifier(tree.DecisionTreeClassifier(), 
                          n_estimators = 24,
                          max_samples = 0.2,
                          bootstrap = True,
                          random_state=seed)


bagg.fit(train_atts[1], train_label[1])
prediction = bagg.predict(test_atts[1])
metrics.accuracy_score(test_label[1], prediction)

0.69480519480519476

Como podemos observar la configuración óptima obtenida ha sido la siguiente:

Configuracion más óptima: 
- Accuracy :  0.798147090551
- Estimator:  DecisionTreeClassifier(
                        class_weight=None, criterion='gini', max_depth=None,
                        max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, presort=False, random_state=None,
                        splitter='best')
- N_estimators:  24
- Max_Samples:  0.2
- Bootstrap:  True
- Random_state:  1234
- Accuracy tras ejecutar test: 0.69480519480519476

El hecho de aumentar de forma muy alta el número de modelos (n_estimators) ha sido para comprobar si se producía alguna mejora, sin embargo nos ha llevado a un gasto de tiempo innecesario dado que la mejor configuración se encuentra en 24 estimadores o modelos.

Podemos observar también las configuraciones candidatas que se van evaluando hasta obtener la más óptima, y de ello obtenemos las siguientes conclusiones:
- Ante un número determinado de estimadores, el tamaño de la muestra juega un papel fundamental en el aumento del accuracy, dado que cuánto mayor sea el conjunto de entrenamiento mejor será nuestro algoritmo.
- Sin embargo, cuando no sea posible mejorar a través del crecimiento de la muestra, será necesario incrementar el número de modelos/estimadores utilizados para conseguir una mejor generalización del conjunto de datos.

A menor tamaño de la muestra a utilizar, menor importancia del bootstrap debido a que la probabilidad de que se repitan es baja, sin embargo a mayor tamaño, mayor penalización de la misma dado que se pierde conocimiento, debido a que aumenta la probabilidad de que se repitan las instancias de la muestra.


In [10]:
clasif=ensemble(train_atts[1],train_label[1], test_atts[1], test_label[1], 60, False,150,seed,20,1)
metrics.accuracy_score(test_label[1],clasif)

0.74025974025974028

En cuanto a nuestro ensemble, obtenemos un accuracy más alto que el anterior. Esto es debido a que KNN como tal no crea un modelo, sino que predice conforme a los datos de train. Por lo tanto, comparar un KNN con un árbol de decisión no debería ser lo más correcto.

### 3.2.1.2 Bagging del dataset Wisconsin

In [11]:
baggingMax(train_atts[2], train_label[2], 3, "accuracy", tree.DecisionTreeClassifier(), 60, 1, seed)

Configuracion candidata: 
accuracy : 0.92 (+/- 0.05)
Estimator:  DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
N_estimators:  1
Max_Samples:  0.1
Bootstrap:  True
Random_state:  1234
Configuracion candidata: 
accuracy : 0.93 (+/- 0.04)
Estimator:  DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
N_estimators:  1
Max_Samples:  0.1
Bootstrap:  False
Random_state:  1234
Configuracion candidata: 

Cogemos los parámetros que han maximizado el score del ensemble y lo ejecutamos a parte con el fin de obtener su Accuracy con respecto al conjunto de test:

In [12]:
bagg = BaggingClassifier(tree.DecisionTreeClassifier(), 
                          n_estimators = 11,
                          max_samples = 0.6,
                          bootstrap = True,
                          random_state=seed)


bagg.fit(train_atts[2], train_label[2])
prediction = bagg.predict(test_atts[2])
metrics.accuracy_score(test_label[2], prediction)

0.95714285714285718

Como podemos observar la configuración óptima obtenida ha sido la siguiente:

Configuracion más óptima: 
- Accuracy :  0.969610718188
- Estimator:  DecisionTreeClassifier(
                        class_weight=None, criterion='gini', max_depth=None,
                        max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, presort=False, random_state=None,
                        splitter='best')
- N_estimators:  11
- Max_Samples:  0.6
- Bootstrap:  True
- Random_state:  1234
- Accuracy tras ejecutar test: 0.95714285714285718

Como podemos observar, en ambos casos (Pima y wisconsin), no es necesario usar demasiados estimadores, sino que la mejora se suele estancar en cuánto a 11 aproximadamente, en el caso de Pima fue necesario más estimadores, por que habría más margen de mejora de Accuracy, en el caso de Wisconsin, con un Accuracy de 0.9696 la mejora es altamente complicada de conseguir.
La explicación del proceso de conseguir la configuración óptima es similar al anterior.


In [13]:
clasif=ensemble(train_atts[2],train_label[2], test_atts[2], test_label[2], 60, False,150,seed,20,1)
metrics.accuracy_score(test_label[2],clasif)

0.92142857142857137

En cuanto a nuestro ensemble, obtenemos un accuracy un poco más bajo que el anterior, además de que no podemos comparar la eficacia de un árbol con un knn. Cómo dijimos anteriormente KNN como tal no crea un modelo, sino que predice conforme a los datos de train. Por lo tanto, comparar un KNN con un árbol de decisión no debería ser lo más correcto.

## 3.2.2 Boosting

In [14]:
def boostingMax(train_atts, train_label, cv, scoring,  estimator, n_estimators, learningRate, random_state):
    maxScore = 0.0
    config = (None, None, None)
    for i in range(1,n_estimators+1):
        j=0.1
        while j<=learningRate:
            
            boost = AdaBoostClassifier(base_estimator=estimator, 
                      n_estimators = i,
                      learning_rate = j,
                      random_state=random_state)
            scores_boost = cross_val_score(boost, train_atts, train_label, cv=cv, scoring=scoring)
            if scores_boost.mean() > maxScore:
                maxScore = scores_boost.mean()
                config = (i,j)
                print("Configuracion candidata: ")
                print(scoring, ": %0.2f (+/- %0.2f)" % (scores_boost.mean(), scores_boost.std()*2))
                print("Estimator: " , estimator)
                print("N_estimators: " , config[0])
                print("Learning_rate: " , config[1])
                print("Random_state: " , random_state)

            j+=0.1
    print("---------------- RESULTADO --------------")
    print("Configuracion mas optima: ")
    print(scoring , ": " , maxScore)
    print("Estimator: " , estimator)
    print("N_estimators: " , config[0])
    print("Learning_rate: " , config[1])
    print("Random_state: " , random_state)

### 3.2.2.1 Boosting del dataset Pima

In [15]:
boostingMax(train_atts[1], train_label[1], 3, "accuracy", tree.DecisionTreeClassifier(), 100, 1, seed)

Configuracion candidata: 
accuracy : 0.74 (+/- 0.04)
Estimator:  DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
N_estimators:  1
Learning_rate:  0.1
Random_state:  1234
---------------- RESULTADO --------------
Configuracion mas optima: 
accuracy :  0.740973412019
Estimator:  DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
N_estimators:  1
Learning_rate:  0.1
Random_state:  1234


La configuración óptima ha sido la siguiente:

Configuracion más óptima: 
- Accuracy :  0.740973412019
- Estimator:  DecisionTreeClassifier(
                        class_weight=None, criterion='gini', max_depth=None,
                        max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, presort=False, random_state=None,
                        splitter='best')
- N_estimators:  1
- Learning_rate:  0.1
- Random_state:  1234

Como podemos observar, usando un solo estimador, ha sido más que suficiente para no obtener mejora, nosotros creemos que esto es debido a que como todos los casos de training han sido clasificados correctamente, no ha sido necesario usar más estimadores. 
El boosting básicamente consiste en asignar pesos a cada ejemplo del conjunto de entrenamiento, se aprende un modelo y para el siguiente clasificador a usar hará más incapié en aquellos casos que no han sido bien clasificados modificando los pesos. 
En nuestro caso, dado que nuestro árbol de decisión no tiene ningún límite de expansión, todos los casos fueron clasificados, sobreajustado a training, por lo tanto esto lo que ha hecho ha sido ejecutar un decisionTree.

In [16]:
boostingMax(train_atts[1], train_label[1], 3, "accuracy", tree.DecisionTreeClassifier(max_depth=5), 100, 1, seed)

Configuracion candidata: 
accuracy : 0.76 (+/- 0.01)
Estimator:  DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
N_estimators:  1
Learning_rate:  0.1
Random_state:  1234
Configuracion candidata: 
accuracy : 0.76 (+/- 0.00)
Estimator:  DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
N_estimators:  2
Learning_rate:  0.1
Random_state:  1234
Configuracion candidata: 
accuracy : 0.76 (+/- 0.01)
Estimator

---------------- RESULTADO --------------
Configuracion mas optima: 
accuracy :  0.794831524843
Estimator:  DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
N_estimators:  98
Learning_rate:  0.8999999999999999
Random_state:  1234


Cogemos los parámetros que han maximizado el score del ensemble y lo ejecutamos a parte con el fin de obtener su Accuracy con respecto al conjunto de test:

In [17]:
boost = AdaBoostClassifier(base_estimator=tree.DecisionTreeClassifier(max_depth=5), 
                      n_estimators = 98,
                      learning_rate = 0.89999,
                      random_state=seed)


boost.fit(train_atts[1], train_label[1])
prediction = boost.predict(test_atts[1])
metrics.accuracy_score(test_label[1], prediction)

0.6428571428571429

En este caso obtenemos la siguiente configuración óptima:

Configuracion más óptima: 
- Accuracy :  0.794831524843
- Estimator:  DecisionTreeClassifier(
                        class_weight=None, criterion='gini', max_depth=5,
                        max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, presort=False, random_state=None,
                        splitter='best')
- N_estimators:  98
- Learning_rate:  0.8999999999999999
- Random_state:  1234
- Accuracy tras ejecutar test: 0.6428571428571429

Este ejemplo ejecutado anteriormente apoya nuestra afirmación.
Como podemos observar al limitar el árbol de decisión, si que tendrá utilidad usar el boosting y vemos que obtenemos un mejor accuracy.
Este caso generaliza mucho mejor que el anterior dado que está limitado, y además podemos ver que learning_rate de los modelos anteriores tiene una buena importancia. Hay muchas veces en las que el learning_rate es bajo, esto es debido a que a veces es necesario partir prácticamente de un modelo nuevo con el fin de seguir clasificando nuevas instancias y no quedarnos estancados en los modelos anteriores que ya no aportan mejores.

In [18]:
clasif=ensemble(train_atts[1],train_label[1], test_atts[1], test_label[1], 100, False,150,seed,20,1)
metrics.accuracy_score(test_label[1],clasif)

0.74675324675324672

En cuanto a nuestro ensemble, obtenemos un accuracy más alto que el anterior. Esto es debido a que KNN como tal no crea un modelo, sino que predice conforme a los datos de train. Por lo tanto, comparar un KNN con un árbol de decisión no debería ser lo más correcto.

### 3.2.2.2 Boosting del dataset Wisconsin

In [19]:
boostingMax(train_atts[2], train_label[2], 3, "accuracy", tree.DecisionTreeClassifier(), 100, 1, seed)

Configuracion candidata: 
accuracy : 0.96 (+/- 0.01)
Estimator:  DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
N_estimators:  1
Learning_rate:  0.1
Random_state:  1234
---------------- RESULTADO --------------
Configuracion mas optima: 
accuracy :  0.955264217124
Estimator:  DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
N_estimators:  1
Learning_rate:  0.1
Random_state:  1234


In [20]:
boostingMax(train_atts[2], train_label[2], 3, "accuracy", tree.DecisionTreeClassifier(max_depth=5), 100, 1, seed)

Configuracion candidata: 
accuracy : 0.95 (+/- 0.02)
Estimator:  DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
N_estimators:  1
Learning_rate:  0.1
Random_state:  1234
Configuracion candidata: 
accuracy : 0.95 (+/- 0.02)
Estimator:  DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
N_estimators:  2
Learning_rate:  0.1
Random_state:  1234
Configuracion candidata: 
accuracy : 0.96 (+/- 0.01)
Estimator

Cogemos los parámetros que han maximizado el score del ensemble y lo ejecutamos a parte con el fin de obtener su Accuracy con respecto al conjunto de test:

In [21]:
boost = AdaBoostClassifier(base_estimator=tree.DecisionTreeClassifier(max_depth=5), 
                      n_estimators = 37,
                      learning_rate = 0.99999,
                      random_state=seed)


boost.fit(train_atts[2], train_label[2])
prediction = boost.predict(test_atts[2])
metrics.accuracy_score(test_label[2], prediction)

0.94999999999999996

Obtenemos la siguiente configuración óptima:

Configuracion más óptima: 
- Accuracy :  0.976750426466
- Estimator:  DecisionTreeClassifier(
                        class_weight=None, criterion='gini', max_depth=5,
                        max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, presort=False, random_state=None,
                        splitter='best')
- N_estimators:  37
- Learning_rate:  0.9999999999999999
- Random_state:  1234
- Accuracy tras ejecutar test: 0.94999999999999996

Como podemos observar al limitar el árbol de decisión, si que tendrá utilidad usar el boosting y vemos que obtenemos un mejor accuracy.
Una vez que obtenemos un modelo que clasifica muy bien nuevos casos y el accuracy ya es muy alto, interesa más mantener un learning_rate alto, dado que el modelo ya es "perfecto" como tal y la mejora no es significativa.
Por lo demás, la explicación es similar a la anterior.

In [22]:
clasif=ensemble(train_atts[2],train_label[2], test_atts[2], test_label[2], 100, False,150,seed,20,1)
metrics.accuracy_score(test_label[2],clasif)

0.92142857142857137

En cuanto a nuestro ensemble, obtenemos un accuracy un poco más bajo que el anterior, además de que no podemos comparar la eficacia de un árbol con un knn. Cómo dijimos anteriormente KNN como tal no crea un modelo, sino que predice conforme a los datos de train. Por lo tanto, comparar un KNN con un árbol de decisión no debería ser lo más correcto.

## 3.2.3 Random Forest

In [23]:
def randomForestMax(train_atts, train_label, cv, scoring, n_estimators, max_depth, min_samples_split, min_samples_leaf, random_state):
    maxScore = 0.0
    config = (None, None, None, None, None, None)
    for i in range(1,n_estimators+1):
        for j in ["gini","entropy"]:
            for k in [None] + [a for a in range(1,max_depth+1)]:
                for l in range(2,min_samples_split+1):
                    for m in range(1,min_samples_leaf+1):
                        for n in ["True","False"]:
                            rf = RandomForestClassifier(n_estimators = i, 
                                                        criterion=j, 
                                                        max_depth=k,
                                                        min_samples_split = l,
                                                        min_samples_leaf = m,
                                                        bootstrap = n,
                                                        random_state = random_state)
                            scores_rf = cross_val_score(rf, train_atts, train_label, cv=cv, scoring=scoring)
                            if scores_rf.mean() > maxScore:
                                maxScore = scores_rf.mean()
                                config = (i,j,k,l,m,n)
                                print("Configuracion candidata: ")
                                print(scoring, ": %0.2f (+/- %0.2f)" % (scores_rf.mean(), scores_rf.std()*2))
                                print("Criterion: " , config[1])
                                print("N_estimators: " , config[0])
                                print("Max_depth: " , config[2])
                                print("Min_samples_split: " , config[3])
                                print("Min_samples_leaf: " , config[4])
                                print("Bootstrap: " , config[5])
                                print("Random_state: " , random_state)

    print("---------------- RESULTADO --------------")
    print("Configuracion mas optima: ")
    print(scoring , ": " , maxScore)
    print("Criterion: " , config[1])
    print("N_estimators: " , config[0])
    print("Max_depth: " , config[2])
    print("Min_samples_split: " , config[3])
    print("Min_samples_leaf: " , config[4])
    print("Bootstrap: " , config[5])
    print("Random_state: " , random_state)

### 3.2.3.1 Random Forest del dataset Pima

In [24]:
randomForestMax(train_atts[1], train_label[1], 3, "accuracy", 50, 5, 4, 3, seed)

Configuracion candidata: 
accuracy : 0.69 (+/- 0.04)
Criterion:  gini
N_estimators:  1
Max_depth:  None
Min_samples_split:  2
Min_samples_leaf:  1
Bootstrap:  True
Random_state:  1234
Configuracion candidata: 
accuracy : 0.74 (+/- 0.07)
Criterion:  gini
N_estimators:  1
Max_depth:  None
Min_samples_split:  2
Min_samples_leaf:  2
Bootstrap:  True
Random_state:  1234
Configuracion candidata: 
accuracy : 0.74 (+/- 0.05)
Criterion:  gini
N_estimators:  2
Max_depth:  None
Min_samples_split:  2
Min_samples_leaf:  2
Bootstrap:  True
Random_state:  1234
Configuracion candidata: 
accuracy : 0.75 (+/- 0.06)
Criterion:  gini
N_estimators:  2
Max_depth:  None
Min_samples_split:  2
Min_samples_leaf:  3
Bootstrap:  True
Random_state:  1234
Configuracion candidata: 
accuracy : 0.76 (+/- 0.02)
Criterion:  gini
N_estimators:  2
Max_depth:  4
Min_samples_split:  2
Min_samples_leaf:  1
Bootstrap:  True
Random_state:  1234
Configuracion candidata: 
accuracy : 0.77 (+/- 0.01)
Criterion:  gini
N_estimators:

Cogemos los parámetros que han maximizado el score del ensemble y lo ejecutamos a parte con el fin de obtener su Accuracy con respecto al conjunto de test:

In [25]:
rf = RandomForestClassifier(n_estimators = 18, 
                                criterion="gini", 
                                max_depth=5,
                                min_samples_split = 2,
                                min_samples_leaf = 3,
                                bootstrap = True,
                                random_state = seed)


rf.fit(train_atts[1], train_label[1])
prediction = rf.predict(test_atts[1])
metrics.accuracy_score(test_label[1], prediction)

0.70779220779220775

La configuración óptima es la siguiente:

Configuracion más óptima: 
- Accuracy :  0.798147090551
- Criterion:  gini
- N_estimators:  18
- Max_depth:  5
- Min_samples_split:  2
- Min_samples_leaf:  3
- Bootstrap:  True
- Random_state:  1234
- Accuracy tras ejecutar test: 0.70779220779220775

Podemos observar las siguientes características del proceso de mejora de la configuración:
- El mejor accuracy se ha obtenido cuando la máxima profundidad ha sido None, esto es debido a que el árbol se ha podido expandir mejor y ha podido clasificar más número de casos de manera correcta y por lo tanto ha obtenido un accuracy mayor en test. Debemos de decir que los modelos son modelos grandes (muy expandidos) y poco prácticos, y preferimos modelos más sencillos (navaja de ockham).
- Luego también cuanto menor sea el criterio de división (min_samples_split), se suele obtener mejor accuracy debido a que el árbol se ramifica más y clasifica casos más concretos.
- En cuánto al min_samples_leaf cuanto mayor sea, más compacto será el árbol generado y generalizará mejor puesto que limitará la capacidad de expansión del nodo hoja.
- El hecho de combinar min_samples_split con min_samples_leaf nos da una alta capacidad de limitación/expansión del árbol, podríamos decir que es una especie de "prepoda".



In [26]:
clasif=ensemble(train_atts[1],train_label[1], test_atts[1], test_label[1], 50, False,150,seed,20,1)
metrics.accuracy_score(test_label[1],clasif)

0.74675324675324672

En cuanto a nuestro ensemble, obtenemos un accuracy más alto que el anterior. Esto es debido a que KNN como tal no crea un modelo, sino que predice conforme a los datos de train. Por lo tanto, comparar un KNN con un árbol de decisión no debería ser lo más correcto.

### 3.2.3.2 Random Forest del dataset Wisconsin

In [27]:
randomForestMax(train_atts[2], train_label[2], 3, "accuracy", 50, 5, 4, 2, seed)

Configuracion candidata: 
accuracy : 0.95 (+/- 0.01)
Criterion:  gini
N_estimators:  1
Max_depth:  None
Min_samples_split:  2
Min_samples_leaf:  1
Bootstrap:  True
Random_state:  1234
Configuracion candidata: 
accuracy : 0.96 (+/- 0.03)
Criterion:  gini
N_estimators:  1
Max_depth:  3
Min_samples_split:  2
Min_samples_leaf:  1
Bootstrap:  True
Random_state:  1234
Configuracion candidata: 
accuracy : 0.96 (+/- 0.02)
Criterion:  entropy
N_estimators:  1
Max_depth:  4
Min_samples_split:  2
Min_samples_leaf:  1
Bootstrap:  True
Random_state:  1234
Configuracion candidata: 
accuracy : 0.96 (+/- 0.02)
Criterion:  entropy
N_estimators:  1
Max_depth:  4
Min_samples_split:  2
Min_samples_leaf:  2
Bootstrap:  True
Random_state:  1234
Configuracion candidata: 
accuracy : 0.96 (+/- 0.03)
Criterion:  gini
N_estimators:  2
Max_depth:  None
Min_samples_split:  2
Min_samples_leaf:  2
Bootstrap:  True
Random_state:  1234
Configuracion candidata: 
accuracy : 0.96 (+/- 0.03)
Criterion:  entropy
N_estimato

Cogemos los parámetros que han maximizado el score del ensemble y lo ejecutamos a parte con el fin de obtener su Accuracy con respecto al conjunto de test:

In [28]:
rf = RandomForestClassifier(n_estimators = 13, 
                                criterion="gini", 
                                max_depth=None,
                                min_samples_split = 3,
                                min_samples_leaf = 1,
                                bootstrap = True,
                                random_state = seed)


rf.fit(train_atts[2], train_label[2])
prediction = rf.predict(test_atts[2])
metrics.accuracy_score(test_label[2], prediction)

0.93571428571428572

Obtenemos la siguiente configuración óptima:

Configuracion más óptima: 
- Accuracy :  0.973175780576
- Criterion:  gini
- N_estimators:  13
- Max_depth:  None
- Min_samples_split:  3
- Min_samples_leaf:  1
- Bootstrap:  True
- Random_state:  1234
- Accuracy tras ejecutar test: 0.93571428571428572

En el caso de wisconsin, podemos ver que la acción elegida es aquella configuración de máxima expansión (max_depth=None, min_samples_split=3, min_samples_leaf=1).
También hemos comprobado que para mejorar el Accuracy de la configuración se elige entre aumentar el número de estimadores que se usa, o aumentar la máxima profundidad posible del árbol.

In [29]:
clasif=ensemble(train_atts[2],train_label[2], test_atts[2], test_label[2], 100, False,150,seed,20,1)
metrics.accuracy_score(test_label[2],clasif)

0.92142857142857137

En cuanto a nuestro ensemble, obtenemos un accuracy un poco más bajo que el anterior, además de que no podemos comparar la eficacia de un árbol con un knn. Cómo dijimos anteriormente KNN como tal no crea un modelo, sino que predice conforme a los datos de train. Por lo tanto, comparar un KNN con un árbol de decisión no debería ser lo más correcto.

## 3.2.4 Gradient Boosting

In [30]:
def gradientBoostingMax(train_atts, train_label, cv, scoring, n_estimators, learningRate, subsample, max_depth, min_samples_split, min_samples_leaf, random_state):
    maxScore = 0.0
    config = (None, None, None, None, None, None)
    for i in range(1,n_estimators+1):
        j=0.1
        while j<=learningRate:
            for k in [None] + [a for a in range(1,max_depth+1)]:
                for l in range(2,min_samples_split+1):
                    for m in range(1,min_samples_leaf+1):
                        n=0.1
                        while n<=subsample:
                            gboost = GradientBoostingClassifier(n_estimators = i, 
                                                        learning_rate=j, 
                                                        max_depth=k,
                                                        min_samples_split = l,
                                                        min_samples_leaf = m,
                                                        subsample = n,
                                                        random_state = random_state)
                            scores_gboost = cross_val_score(gboost, train_atts, train_label, cv=cv, scoring=scoring)
                            if scores_gboost.mean() > maxScore:
                                maxScore = scores_gboost.mean()
                                config = (i,j,k,l,m,n)
                                print("Configuracion candidata: ")
                                print(scoring, ": %0.2f (+/- %0.2f)" % (scores_gboost.mean(), scores_gboost.std()*2))
                                print("Learning_rate: " , config[1])
                                print("N_estimators: " , config[0])
                                print("Max_depth: " , config[2])
                                print("Min_samples_split: " , config[3])
                                print("Min_samples_leaf: " , config[4])
                                print("Subsample: " , config[5])
                                print("Random_state: " , random_state)
                            n+=0.3
            j+=0.3

    print("---------------- RESULTADO --------------")
    print("Configuracion mas optima: ")
    print(scoring , ": " , maxScore)
    print("Learning_rate: " , config[1])
    print("N_estimators: " , config[0])
    print("Max_depth: " , config[2])
    print("Min_samples_split: " , config[3])
    print("Min_samples_leaf: " , config[4])
    print("Subsample: " , config[5])
    print("Random_state: " , random_state)

### 3.2.4.1 Gradient Boosting del dataset Pima

In [31]:
gradientBoostingMax(train_atts[1], train_label[1], 3, "accuracy", 25, 1, 1, 5, 4, 2, seed)

Configuracion candidata: 
accuracy : 0.65 (+/- 0.00)
Learning_rate:  0.1
N_estimators:  1
Max_depth:  None
Min_samples_split:  2
Min_samples_leaf:  1
Subsample:  0.1
Random_state:  1234
Configuracion candidata: 
accuracy : 0.69 (+/- 0.04)
Learning_rate:  0.4
N_estimators:  1
Max_depth:  None
Min_samples_split:  2
Min_samples_leaf:  1
Subsample:  0.1
Random_state:  1234
Configuracion candidata: 
accuracy : 0.73 (+/- 0.03)
Learning_rate:  0.4
N_estimators:  1
Max_depth:  None
Min_samples_split:  2
Min_samples_leaf:  1
Subsample:  0.4
Random_state:  1234
Configuracion candidata: 
accuracy : 0.74 (+/- 0.03)
Learning_rate:  0.4
N_estimators:  1
Max_depth:  None
Min_samples_split:  2
Min_samples_leaf:  1
Subsample:  1.0
Random_state:  1234
Configuracion candidata: 
accuracy : 0.76 (+/- 0.04)
Learning_rate:  0.4
N_estimators:  1
Max_depth:  None
Min_samples_split:  2
Min_samples_leaf:  2
Subsample:  1.0
Random_state:  1234
Configuracion candidata: 
accuracy : 0.76 (+/- 0.01)
Learning_rate:  0

Cogemos los parámetros que han maximizado el score del ensemble y lo ejecutamos a parte con el fin de obtener su Accuracy con respecto al conjunto de test:

In [32]:
gBoost = GradientBoostingClassifier(n_estimators = 16, 
                                    learning_rate=0.4, 
                                    max_depth=None,
                                    min_samples_split = 2,
                                    min_samples_leaf = 2,
                                    subsample = 1.0,
                                    random_state = seed)


gBoost.fit(train_atts[1], train_label[1])
prediction = gBoost.predict(test_atts[1])
metrics.accuracy_score(test_label[1], prediction)

0.68181818181818177

Obtenemos la siguiente configuración óptima:

Configuracion más óptima: 
- Accuracy :  0.794815660892
- Learning_rate:  0.4
- N_estimators:  16
- Max_depth:  None
- Min_samples_split:  2
- Min_samples_leaf:  2
- Subsample:  1.0
- Random_state:  1234
- Accuracy tras ejecutar test: 0.68181818181818177

Con respecto a los parámetros similares al de randomForest la explicación es similar al randomForest de Wisconsin, dado que generamos un árbol sin limite de expansión (o con un límite muy bajo: max_depth: None, min_samples_split: 2 y min_samples_leaf: 2).

En cuanto al learning_rate el promedio es 0.5 puesto que nos interesa obtener estimadores que no estén basados al 100% en en el anterior, sino que queremos mejorar.

Y en cuanto al subsample, cuanto menor es, más reduce la varianza e incrementa el sesgo. En este caso, se prefiere un mayor subsample debido a que preferimos unos datos más variados (que generalicen mejor) y con menor sesgo (que estén más cerca de la predicción esperada).

In [33]:
clasif=ensemble(train_atts[1],train_label[1], test_atts[1], test_label[1], 100, False,150,seed,20,1)
metrics.accuracy_score(test_label[1],clasif)

0.74675324675324672

En cuanto a nuestro ensemble, obtenemos un accuracy más alto que el anterior. Esto es debido a que KNN como tal no crea un modelo, sino que predice conforme a los datos de train. Por lo tanto, comparar un KNN con un árbol de decisión no debería ser lo más correcto.

### 3.2.4.2 Gradient Boosting del dataset Wisconsin

In [34]:
gradientBoostingMax(train_atts[2], train_label[2], 3, "accuracy", 25, 1, 1, 5, 4, 2, seed)

Configuracion candidata: 
accuracy : 0.65 (+/- 0.00)
Learning_rate:  0.1
N_estimators:  1
Max_depth:  None
Min_samples_split:  2
Min_samples_leaf:  1
Subsample:  0.1
Random_state:  1234
Configuracion candidata: 
accuracy : 0.93 (+/- 0.03)
Learning_rate:  0.4
N_estimators:  1
Max_depth:  None
Min_samples_split:  2
Min_samples_leaf:  1
Subsample:  0.1
Random_state:  1234
Configuracion candidata: 
accuracy : 0.94 (+/- 0.02)
Learning_rate:  0.4
N_estimators:  1
Max_depth:  None
Min_samples_split:  2
Min_samples_leaf:  1
Subsample:  0.4
Random_state:  1234
Configuracion candidata: 
accuracy : 0.95 (+/- 0.03)
Learning_rate:  0.4
N_estimators:  1
Max_depth:  None
Min_samples_split:  2
Min_samples_leaf:  1
Subsample:  1.0
Random_state:  1234
Configuracion candidata: 
accuracy : 0.95 (+/- 0.01)
Learning_rate:  0.4
N_estimators:  1
Max_depth:  5
Min_samples_split:  2
Min_samples_leaf:  1
Subsample:  1.0
Random_state:  1234
Configuracion candidata: 
accuracy : 0.96 (+/- 0.01)
Learning_rate:  0.7


Cogemos los parámetros que han maximizado el score del ensemble y lo ejecutamos a parte con el fin de obtener su Accuracy con respecto al conjunto de test:

In [35]:
gBoost = GradientBoostingClassifier(n_estimators = 15, 
                                    learning_rate=0.4, 
                                    max_depth=None,
                                    min_samples_split = 4,
                                    min_samples_leaf = 1,
                                    subsample = 0.7,
                                    random_state = seed)


gBoost.fit(train_atts[2], train_label[2])
prediction = gBoost.predict(test_atts[2])
metrics.accuracy_score(test_label[2], prediction)

0.94285714285714284

Obtenemos la siguiente configuración óptima:

Configuracion más óptima: 
- Accuracy :  0.973175780576
- Learning_rate:  0.4
- N_estimators:  15
- Max_depth:  None
- Min_samples_split:  4
- Min_samples_leaf:  1
- Subsample:  0.7
- Random_state:  1234
- Accuracy tras ejecutar test: 0.94285714285714284

Con respecto a los parámetros similares al de randomForest la explicación es similar al randomForest de Wisconsin, dado que generamos un árbol sin limite de expansión (o con un límite muy bajo: max_depth: None, min_samples_split: 4 y min_samples_leaf: 1), aunque en este caso se podría pensar que min_samples_split limita bastante, pero el hecho de tener max_depth = None, esta limitación es escasa, además que si observamos las configuraciones subóptimas a esta, podemos ver que dichos parámetros no han obtenido una gran diferencia en cuanto a la mejora obtenida.

En cuanto al learning_rate el promedio es 0.5 puesto que nos interesa obtener estimadores que no estén basados al 100% en en el anterior, sino que queremos mejorar.

Y en cuanto al subsample, cuanto menor es, más reduce la varianza e incrementa el sesgo. En este caso, se prefiere un mayor subsample debido a que preferimos unos datos más variados (que generalicen mejor) y con menor sesgo (que estén más cerca de la predicción esperada).

In [36]:
clasif=ensemble(train_atts[2],train_label[2], test_atts[2], test_label[2], 100, False,150,seed,20,1)
metrics.accuracy_score(test_label[2],clasif)

0.92142857142857137

En cuanto a nuestro ensemble, obtenemos un accuracy un poco más bajo que el anterior, además de que no podemos comparar la eficacia de un árbol con un knn. Cómo dijimos anteriormente KNN como tal no crea un modelo, sino que predice conforme a los datos de train. Por lo tanto, comparar un KNN con un árbol de decisión no debería ser lo más correcto.

## 3.3 Resultados obtenidos tras usar Bagging, Boosting, Random Forest, Gradient Boosting y nuestro Ensemble

Con el dataset Pima, los Accuracies obtenidos tras testear con test han sido los siguientes:

In [37]:
results = (("Nuestro ensemble", 0.70779220779220775), 
           ("Bagging", 0.69480519480519476), 
           ("Random Forest", 0.70779220779220775),
           ("Boosting", 0.6428571428571429),
           ("Gradient Boosting", 0.68181818181818177))

printTable(results)

Nuestro ensemble,0.7077922077922078
Bagging,0.6948051948051948
Random Forest,0.7077922077922078
Boosting,0.6428571428571429
Gradient Boosting,0.6818181818181818


Con el dataset Wisconsin, los Accuracies obtenidos tras testear con test han sido los siguiente:

In [38]:
results = (("Nuestro ensemble", 0.94999999999999996), 
           ("Bagging", 0.95714285714285718), 
           ("Random Forest", 0.93571428571428572),
           ("Boosting", 0.94999999999999996),
           ("Gradient Boosting", 0.94285714285714284))

printTable(results)

Nuestro ensemble,0.95
Bagging,0.9571428571428572
Random Forest,0.9357142857142857
Boosting,0.95
Gradient Boosting,0.9428571428571428


Como podemos observar, sorprendentemente nuestro ensemble se ha posicionado entre los primeros en obtener una mejor predicción. En cuanto a los ensembles implementados en Scikit, podemos observar una gran diversidad de resultados, siendo altamente complicado establecer un orden general de quién es mejor con respecto a los demás. Esto último, está altamente condicionado con el dataset, siendo en algunos casos unos ensembles más importantes que otros.

# 4. Selección de variables

## 4.1 Método filter basado en rankings y la importancia de las variables para evaluar distintos subconjuntos

### 4.1.1 Método filter basado en rankings

In [39]:
# Función que retorna el ranking de las variables de un determinado dataset
def filterRank(metrica, df):
    # Obtiene los scores de aplicar una determianda metríca al conjunto de datos
    scores = list(metrica)
    names = list(df.keys())
    # Se obtiene una lista ranks que junta los scores con su respectiva variable predictora
    ranks = sorted( list(zip(scores, names)), reverse=True )
    return ranks

# Función que retorna un par de diccionarios (train_atts y test_atts) eliminando las variables con menor Score,
# se indicará por parámetro la métrica a emplear, el número de variables a salvar, el dataset completo [solo usado
# para obtener el conjunto de variables predictoras], train y test
def selectK(metrica, k, df, train_atts, train_label, test_atts):
    # Se obtiene una lista indicando las variables a salvar con respecto a los parámetros dados
    # Nota: fss.get_support se obtiene una lista con True's y False's indicando que variables se salvan o no, están
    # en orden con respecto a train_atts
    fss = SelectKBest(metrica, k=k).fit(train_atts, train_label)
    train_attsCopy = train_atts.copy()
    test_attsCopy = test_atts.copy()
    print("Variables seleccionadas: ")
    # Bucle que mostrará las variables seleccionadas para salvar y eliminará el resto (esto se hace tanto en train
    # como en test, se realiza con test el fin de evitar errores a la hora de predecir con éste)
    for i in range(0,len(fss.get_support())):
        if (fss.get_support())[i]:
            print((df.keys())[i])
        else:
            del train_attsCopy[(df.keys())[i]]
            del test_attsCopy[(df.keys())[i]]
    print(train_attsCopy.shape)
    return (train_attsCopy, test_attsCopy)

#### 4.1.1.1 Método filter basado en rankings con dataset Pima

Ejecutando el algoritmo de filter basado en rankings, vamos a obtener el ranking de las variables ordenado por su importancia, y posteriormente procederemos a obtener un nuevo diccionario con el que nos quedaremos con solo un número determinado de variables.

In [40]:
metrica=mutual_info_classif(train_atts[1], train_label[1], random_state=seed)
filterRank(metrica, df[1])

[(0.1288987268794517, 'plas'),
 (0.094889792970892284, 'mass'),
 (0.067196685611855322, 'age'),
 (0.064457008631584101, 'preg'),
 (0.040198824529407595, 'pres'),
 (0.035925099264167093, 'insu'),
 (0.014890438929848759, 'skin'),
 (0.0, 'pedi')]

In [41]:
# Obtenemos train y test _atts con las mejores variables, se usará más adelante en los ensembles
(train_attsVar, test_attsVar) = selectK(mutual_info_classif, 4, df[1], train_atts[1], train_label[1], test_atts[1])
print(train_attsVar)
print(test_attsVar)

Variables seleccionadas: 
preg
plas
mass
age
(614, 4)
     preg   plas       mass  age
54      0   74.0  27.800000   22
156     8  183.0  23.300000   32
38      1  108.0  27.100000   24
646     2  101.0  21.800000   22
447     4   97.0  28.200000   22
759     1  109.0  25.400000   21
576     0  101.0  35.700000   26
660     4  189.0  28.500000   37
431     2   71.0  28.000000   22
568     5  155.0  38.700000   34
486     8  124.0  28.700000   52
388     9  145.0  30.300000   53
539     6  102.0  30.800000   36
355     5  109.0  36.000000   60
51      6  125.0  30.000000   32
525     0  139.0  22.100000   21
421     0  113.0  31.000000   21
557     9  184.0  30.000000   49
691    10  108.0  32.400000   42
395     6  125.0  27.600000   49
61      6  137.0  24.200000   55
598     1  130.0  25.900000   22
410     1  108.0  35.500000   24
327     7  196.0  39.800000   41
122     1   90.0  25.100000   25
430     2   92.0  31.600000   24
417     3  108.0  26.000000   25
66     13  106.0  36.6

Se han elegido las 4 mejores variables dado que tenemos 8 variables predictoras en total, y no queríamos usar menos para no dar pie a perder mucha información.
Ahora haremos una primera prueba con nuestro ensemble

In [42]:
clasif=ensemble(train_attsVar,train_label[1], test_attsVar, test_label[1], 100,True,150,seed,5,0.5)
metrics.accuracy_score(test_label[1],clasif)

0.72727272727272729

- Resultado obtenido sin aplicar filter: 0.70779220779220775
- Resultado obtenido aplicando filter: 0.72727272727272729

En el caso de Pima mejora, en parte tiene su lógica debido a que el accuracy era bajo, por ello es sencillo mejorarlo. 
Al elegir las variables que tienen mayor ganancia obtenemos una mejor predicción.

Ahora vamos a ver la influencia que tiene al aplicar filter.
Para ello vamos a ver si obtenemos mejora con respecto al clasificador que ha obtenido uno de los mejores accuracys y uno de los peores.

Uno de los mejores accuracy Pima (RandomForest):

Configuracion más óptima: 
- Accuracy :  0.798147090551
- Criterion:  gini
- N_estimators:  18
- Max_depth:  5
- Min_samples_split:  2
- Min_samples_leaf:  3
- Bootstrap:  True
- Random_state:  1234
- Accuracy de test: 0.70779220779220775

Uno de los peores accuracy Pima (Boosting):

Configuracion más óptima: 
- Accuracy :  0.794831524843
- Estimator:  DecisionTreeClassifier(
                        class_weight=None, criterion='gini', max_depth=5,
                        max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, presort=False, random_state=None,
                        splitter='best')
- N_estimators:  98
- Learning_rate:  0.8999999999999999
- Random_state:  1234
- Accuracy de test: 0.6428571428571429

In [43]:
rf = RandomForestClassifier(n_estimators = 18, 
                                criterion="gini", 
                                max_depth=5,
                                min_samples_split = 2,
                                min_samples_leaf = 3,
                                bootstrap = True,
                                random_state = seed)


rf.fit(train_attsVar, train_label[1])
prediction = rf.predict(test_attsVar)
metrics.accuracy_score(test_label[1], prediction)

0.68181818181818177

En el caso del mejor Accuracy de test, empeoramos dado que perdemos información eliminando las variables de menor importancia, además mejorar una de las mejores configuraciones es más difícil.

In [44]:
boost = AdaBoostClassifier(base_estimator=tree.DecisionTreeClassifier(max_depth=5), 
                      n_estimators = 98,
                      learning_rate = 0.89999,
                      random_state=seed)


boost.fit(train_attsVar, train_label[1])
prediction = boost.predict(test_attsVar)
metrics.accuracy_score(test_label[1], prediction)

0.67532467532467533

En cuanto al de peor configuración, obtenemos una mejora, esto es debido dado que al quedarnos con las variables de más importancia, obtenemos una mejor predicción, además de que mejorar la peor configuración es más sencillo.

#### 4.1.1.2 Método filter basado en rankings con dataset Wisconsin

In [45]:
metrica=mutual_info_classif(train_atts[2], train_label[2], random_state=seed)
filterRank(metrica, df[2])

[(0.48603098768879649, 'cellSize'),
 (0.46993766833372019, 'CellShape'),
 (0.41167880380619892, 'bareNuclei'),
 (0.37911994505258395, 'blandChromatin'),
 (0.35302212105383335, 'epithelialSize'),
 (0.31501779705255006, 'clumpThickness'),
 (0.31169836388044914, 'normalNucleoli'),
 (0.30767797478207859, 'marginalAdhesion'),
 (0.14952150918842766, 'mitoses')]

In [46]:
# Obtenemos train y test _atts con las mejores variables, se usará más adelante en los ensembles
(train_attsVar2, test_attsVar2) = selectK(mutual_info_classif, 4, df[2], train_atts[2], train_label[2], test_atts[2])
print(train_attsVar2)
print(test_attsVar2)

Variables seleccionadas: 
cellSize
CellShape
bareNuclei
blandChromatin
(559, 4)
     cellSize  CellShape  bareNuclei  blandChromatin
91          1          1    1.000000               3
200        10          7    3.536732               8
677         1          1    1.000000               1
234        10         10   10.000000               7
221         1          3    2.000000               2
253         1          1    1.000000               2
102         3          1    2.000000               5
575        10          7    1.000000              10
324         1          1    1.000000               3
445        10         10   10.000000               6
338         1          1    1.000000               2
348         1          1    1.000000               2
295         1          1    1.000000               2
245         3          5   10.000000               7
556         1          1    1.000000               2
688         2          3    1.000000               1
439         1      

En este caso elegimos las mejores 4 variables por la misma explicación de antes.
Ahora haremos una prueba con nuestro ensemble.

In [47]:
clasif=ensemble(train_attsVar2,train_label[2], test_attsVar2, test_label[2], 100,True,150,seed,5,0.5)
metrics.accuracy_score(test_label[2],clasif)

0.94285714285714284

- Resultado obtenido sin aplicar filter: 0.94999999999999996
- Resultado obtenido aplicando filter: 0.94285714285714284

En este caso, dado que la primera prediccion sin utilizar filter es muy buena, lo que estamos haciendo al eliminar las variables de menor ganancia, es perder conocimiento específico.

Ahora vamos a ver la influencia que tiene aplicar filter.
Para ello vamos a ver si obtenemos mejora con respecto al clasificador que ha obtenido el mejor accuracy y el peor.

Uno de los mejores accuracy Wisconsin (Bagging):

Configuracion más óptima: 
- Accuracy :  0.969610718188
- Estimator:  DecisionTreeClassifier(
                        class_weight=None, criterion='gini', max_depth=None,
                        max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, presort=False, random_state=None,
                        splitter='best')
- N_estimators:  11
- Max_Samples:  0.6
- Bootstrap:  True
- Random_state:  1234
- Accuracy de test: 0.95714285714285718


Uno de los peores accuracy Wisconsin (RandomForest):

Configuracion más óptima: 
- Accuracy :  0.973175780576
- Criterion:  gini
- N_estimators:  13
- Max_depth:  None
- Min_samples_split:  3
- Min_samples_leaf:  1
- Bootstrap:  True
- Random_state:  1234  
- Accuracy de test: 0.93571428571428572

In [48]:
bagg = BaggingClassifier(tree.DecisionTreeClassifier(), 
                          n_estimators = 11,
                          max_samples = 0.6,
                          bootstrap = True,
                          random_state=seed)


bagg.fit(train_attsVar2, train_label[2])
prediction = bagg.predict(test_attsVar2)
metrics.accuracy_score(test_label[2], prediction)

0.90714285714285714

In [49]:
rf = RandomForestClassifier(n_estimators = 13, 
                                criterion="gini", 
                                max_depth=None,
                                min_samples_split = 3,
                                min_samples_leaf = 1,
                                bootstrap = True,
                                random_state = seed)


rf.fit(train_attsVar2, train_label[2])
prediction = rf.predict(test_attsVar2)
metrics.accuracy_score(test_label[2], prediction)

0.93571428571428572

En estos casos, dado que obtenemos un accuracy tan alto en las distintas configuraciones (ya sea el más alto o el más bajo), es muy difícil mejorarlo, por lo tanto al eliminar las variables menos importantes, perdemos información.

### 4.1.2 Método filter basado en la importancia de las variables

In [50]:
# Función que retorna un ranking con la importancia de cada una de las variables al aplicar un determinado
# para ello se el indica el clasificador y df [solo usado para obtener las variables predictoras]
def filterImportance(classifer, df):
    scores = classifier.feature_importances_
    names = list(df.keys())
    ranks = sorted( list(zip(scores, names)), reverse=True )
    # Se retorna una lista con las variables y su Score
    return ranks

# Función que retorna un par de diccionarios (train_atts y test_atts) eliminando las variables con menor Score,
# se indicará por parámetro el modelo a emplear, forma de evaluar la importancia de las variables [threshold], 
# el dataset completo [solo usado para obtener el conjunto de variables predictoras], train y test
def selectModel(model,threshold,train_atts, train_label, test_atts, df):
    # Se obtienen las variables a salvar tras aplicar el método SelectFromModel y los parámetros datos por el 
    # usuario
    # Nota: fss.get_support se obtiene una lista con True's y False's indicando que variables se salvan o no, están
    # en orden con respecto a train_atts
    fss = SelectFromModel(estimator = model, threshold = threshold).fit(train_atts, train_label)
    train_attsCopy = train_atts.copy()
    test_attsCopy = test_atts.copy()
    print("Variables seleccionadas: ")
    # Se mira que variables se salva, las que se salvan se muestran en pantalla, las que no se eliminan de train
    # y test (se usa también test para evitar problemas a la hora de usar predicción más adelante)
    for i in range(0,len(fss.get_support())):
        if (fss.get_support())[i]:
            print((df.keys())[i])
        else:
            del train_attsCopy[(df.keys())[i]]
            del test_attsCopy[(df.keys())[i]]
    print(train_attsCopy.shape)
    return (train_attsCopy, test_attsCopy)

#### 4.1.2.1 Método filter basado en la importancia de las variables con dataset Pima

Ejecutando el algoritmo de filter basado en importancia de variables, vamos a obtener la lista de las variables ordenado por su importancia. Para ello, usaremos un clasificador y nos dirá cuales son las variables con la mayor importancia.

En este caso, dado que nuestro ensemble únicamente retorna un vector con todos los casos clasificados, no podemos devolver un modelo, y por lo tanto no podemos ejecutar este método filter. Por ello, vamos a proceder a ejecutar una de las mejores y una de las peores configuraciones y veremos si hay mejora o no.

Ahora vamos a ver la influencia que tiene aplicar filter.
Para ello vamos a ver si obtenemos mejora con respecto al clasificador que ha obtenido uno de los mejores accuracys y uno de los peores.

Uno de los mejores accuracy Pima (RandomForest):

Configuracion más óptima: 
- Accuracy :  0.798147090551
- Criterion:  gini
- N_estimators:  18
- Max_depth:  5
- Min_samples_split:  2
- Min_samples_leaf:  3
- Bootstrap:  True
- Random_state:  1234
- Accuracy de test: 0.70779220779220775

Uno de los peores accuracy Pima (Boosting):

Configuracion más óptima: 
- Accuracy :  0.794831524843
- Estimator:  DecisionTreeClassifier(
                        class_weight=None, criterion='gini', max_depth=5,
                        max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, presort=False, random_state=None,
                        splitter='best')
- N_estimators:  98
- Learning_rate:  0.8999999999999999
- Random_state:  1234
- Accuracy de test: 0.6428571428571429

In [51]:
# Modelo
rf = RandomForestClassifier(n_estimators = 18, 
                                criterion="gini", 
                                max_depth=5,
                                min_samples_split = 2,
                                min_samples_leaf = 3,
                                bootstrap = True,
                                random_state = seed)


classifier = rf.fit(train_atts[1], train_label[1])
ranks = filterImportance(classifier, df[1])
print(ranks)

[(0.31522276881552447, 'plas'), (0.22635369791194165, 'mass'), (0.17287380555691637, 'age'), (0.076229757919770116, 'pedi'), (0.061763106639289252, 'insu'), (0.061046738762102454, 'preg'), (0.050701348379060464, 'pres'), (0.035808776015395227, 'skin')]


In [52]:
(train_attsVar, test_attsVar) = selectModel(rf,"mean",train_atts[1], train_label[1], test_atts[1], df[1])
print (train_attsVar)
print (test_attsVar)

Variables seleccionadas: 
plas
mass
age
(614, 3)
      plas       mass  age
54    74.0  27.800000   22
156  183.0  23.300000   32
38   108.0  27.100000   24
646  101.0  21.800000   22
447   97.0  28.200000   22
759  109.0  25.400000   21
576  101.0  35.700000   26
660  189.0  28.500000   37
431   71.0  28.000000   22
568  155.0  38.700000   34
486  124.0  28.700000   52
388  145.0  30.300000   53
539  102.0  30.800000   36
355  109.0  36.000000   60
51   125.0  30.000000   32
525  139.0  22.100000   21
421  113.0  31.000000   21
557  184.0  30.000000   49
691  108.0  32.400000   42
395  125.0  27.600000   49
61   137.0  24.200000   55
598  130.0  25.900000   22
410  108.0  35.500000   24
327  196.0  39.800000   41
122   90.0  25.100000   25
430   92.0  31.600000   24
417  108.0  26.000000   25
66   106.0  36.600000   45
160  136.0  28.300000   42
590  135.0  40.600000   26
..     ...        ...  ...
86   188.0  32.000000   22
642  122.0  23.000000   40
494   97.0  18.200000   21
148  1

In [53]:
rf.fit(train_attsVar, train_label[1])
prediction = rf.predict(test_attsVar)
metrics.accuracy_score(test_label[1], prediction)

0.69480519480519476

En el caso del mejor Accuracy de test, empeoramos dado que perdemos información eliminando las variables de menor importancia, además mejorar una de las mejores configuraciones es más difícil.

Ahora, vamos a comprobar que ocurre con uno de los que peor Accuracy han obtenido.

In [54]:
# Modelo
boost = AdaBoostClassifier(base_estimator=tree.DecisionTreeClassifier(max_depth=5), 
                      n_estimators = 98,
                      learning_rate = 0.89999,
                      random_state=seed)


classifier = boost.fit(train_atts[1], train_label[1])
ranks = filterImportance(classifier, df[1])
print(ranks)

[(0.18175892439044469, 'pedi'), (0.18005259886452382, 'plas'), (0.15669191820380915, 'mass'), (0.13925013895184199, 'age'), (0.13109894414090861, 'pres'), (0.085254529404819077, 'preg'), (0.075250414579007954, 'skin'), (0.050642531464644709, 'insu')]


In [55]:
(train_attsVar, test_attsVar) = selectModel(boost,"mean",train_atts[1], train_label[1], test_atts[1], df[1])
print (train_attsVar)
print (test_attsVar)

Variables seleccionadas: 
plas
pres
mass
pedi
age
(614, 5)
      plas        pres       mass   pedi  age
54    74.0   52.000000  27.800000  0.269   22
156  183.0   64.000000  23.300000  0.672   32
38   108.0   88.000000  27.100000  0.400   24
646  101.0   58.000000  21.800000  0.155   22
447   97.0   60.000000  28.200000  0.443   22
759  109.0   60.000000  25.400000  0.947   21
576  101.0   76.000000  35.700000  0.198   26
660  189.0  110.000000  28.500000  0.680   37
431   71.0   70.000000  28.000000  0.586   22
568  155.0   84.000000  38.700000  0.619   34
486  124.0   76.000000  28.700000  0.687   52
388  145.0   88.000000  30.300000  0.771   53
539  102.0   82.000000  30.800000  0.180   36
355  109.0   75.000000  36.000000  0.546   60
51   125.0   68.000000  30.000000  0.464   32
525  139.0   62.000000  22.100000  0.207   21
421  113.0   80.000000  31.000000  0.874   21
557  184.0   85.000000  30.000000  1.213   49
691  108.0   66.000000  32.400000  0.272   42
395  125.0   78.00000

In [56]:
boost.fit(train_attsVar, train_label[1])
prediction = boost.predict(test_attsVar)
metrics.accuracy_score(test_label[1], prediction)

0.7142857142857143

En cuanto al de peor configuración, obtenemos una mejora, esto es debido dado que al quedarnos con las variables de más importancia, obtenemos una mejor predicción, además de que mejorar la peor configuración es más sencillo.

#### 4.1.2.2 Método filter basado en la importancia de las variables con dataset Wisconsin

Ejecutando el algoritmo de filter basado en importancia de variables, vamos a obtener la lista de las variables ordenado por su importancia. Para ello, usaremos un clasificador y nos dirá cuales son las variables con la mayor importancia.

En este caso, dado que nuestro ensemble únicamente retorna un vector con todos los casos clasificados, no podemos devolver un modelo, y por lo tanto no podemos ejecutar este método filter. Por ello, vamos a proceder a ejecutar una de las mejores y una de las peores configuraciones y veremos si hay mejora o no.

Ahora vamos a ver la influencia que tiene aplicar filter.
Para ello vamos a ver si obtenemos mejora con respecto al clasificador que ha obtenido el mejor accuracy y el peor.

Uno de los mejores accuracy Wisconsin (Bagging):

Configuracion más óptima: 
- Accuracy :  0.969610718188
- Estimator:  DecisionTreeClassifier(
                        class_weight=None, criterion='gini', max_depth=None,
                        max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, presort=False, random_state=None,
                        splitter='best')
- N_estimators:  11
- Max_Samples:  0.6
- Bootstrap:  True
- Random_state:  1234
- Accuracy de test: 0.95714285714285718


Uno de los peores accuracy Wisconsin (RandomForest):

Configuracion más óptima: 
- Accuracy :  0.973175780576
- Criterion:  gini
- N_estimators:  13
- Max_depth:  None
- Min_samples_split:  3
- Min_samples_leaf:  1
- Bootstrap:  True
- Random_state:  1234
- Accuracy de test: 0.93571428571428572

In [57]:
# Modelo
#bagg = BaggingClassifier(tree.DecisionTreeClassifier(), 
#                          n_estimators = 11,
#                          max_samples = 0.6,
#                          bootstrap = True,
#                          random_state=seed)


#classifier = bagg.fit(train_atts[2], train_label[2])
#ranks = filterImportance(classifier, df[2])
#print(ranks)

In [58]:
#(train_attsVar2, test_attsVar2) = selectModel(bagg,"mean",train_atts[2], train_label[2], test_atts[2], df[2])
#print (train_attsVar2)
#print (test_attsVar2)

In [59]:
#bagg.fit(train_attsVar2, train_label[2])
#prediction = bagg.predict(test_attsVar2)
#metrics.accuracy_score(test_label[2], prediction)

En este caso, dado que Bagging puede ser usado con muchos estimadores base, no presenta el atributo "features_importances", y por lo tanto, no puede ser ejectado.

Ahora, veremos que ocurre con la configuración que obtiene un peor Accuracy:

In [60]:
# Modelo
rf = RandomForestClassifier(n_estimators = 13, 
                                criterion="gini", 
                                max_depth=None,
                                min_samples_split = 3,
                                min_samples_leaf = 1,
                                bootstrap = True,
                                random_state = seed)


classifier = rf.fit(train_atts[2], train_label[2])
ranks = filterImportance(classifier, df[2])
print(ranks)

[(0.34851923334527224, 'cellSize'), (0.15578243752169424, 'CellShape'), (0.14212707564938676, 'blandChromatin'), (0.13831324529848016, 'bareNuclei'), (0.13224663913558732, 'epithelialSize'), (0.031887936819746526, 'clumpThickness'), (0.031693204632165455, 'normalNucleoli'), (0.014574475373896202, 'marginalAdhesion'), (0.0048557522237710768, 'mitoses')]


In [61]:
(train_attsVar2, test_attsVar2) = selectModel(rf,"mean",train_atts[2], train_label[2], test_atts[2], df[2])
print (train_attsVar2)
print (test_attsVar2)

Variables seleccionadas: 
cellSize
CellShape
epithelialSize
bareNuclei
blandChromatin
(559, 5)
     cellSize  CellShape  epithelialSize  bareNuclei  blandChromatin
91          1          1               2    1.000000               3
200        10          7               6    3.536732               8
677         1          1               2    1.000000               1
234        10         10               8   10.000000               7
221         1          3               2    2.000000               2
253         1          1               2    1.000000               2
102         3          1               2    2.000000               5
575        10          7               7    1.000000              10
324         1          1               2    1.000000               3
445        10         10               6   10.000000               6
338         1          1               3    1.000000               2
348         1          1               2    1.000000               2
295     

In [62]:
rf.fit(train_attsVar2, train_label[2])
prediction = rf.predict(test_attsVar2)
metrics.accuracy_score(test_label[2], prediction)

0.9285714285714286

En el caso del mejor Accuracy de test, empeoramos dado que perdemos información eliminando las variables de menor importancia, además mejorar una de las mejores configuraciones es más difícil.

## 4.2 Algoritmo de búsqueda recursiva wrapper

Procedemos a implementar un método wrapper de búsqueda recursiva.
En nuestro caso hemos decidido implementar la selección forward, dado que sus características encajan más con nuestras necesidades:
- Funciona mejor cuando el subconjunto óptimo tiene pocas variables.
- Evalúa conjuntos mucho más pequeños, por lo que es más rápido.
- Tiende a evaluar menos subconjuntos.

In [63]:
def forward(atts,label,seed):
    # Función encargada de retornar un diccionario en el que se eliminan las variables predictoras que no se
    # encuentran en la lista "aSalvar"
    def attsConAtributos(atts,aSalvar):
        # Nos guardamos una copia de atts y trabajamos con attsCopy
        attsCopy = atts.copy()
        # Por cada variable predictora...
        for i in atts.keys():
            # Comprobaoms si no se encuentra en "aSalvar"
            # Si no se encuentra, se elimina
            if i not in aSalvar:
                del attsCopy[i]
        return attsCopy
    
    # Lista que almacena las variables predictoras elegidas
    S = []
    # Lista con todas las variables predictoras del dataset
    X = list(atts.keys())
    # Inicializacio de variables, al comienzo, dado que S es vacía, se considera que el score es de 0.
    puntuacionS = 0
    nuevaPuntuacion = 0
    puntuacionSX = 0
    # Daremos vueltas hasta que la nueva puntuacion y la puntuacion del sistema sean iguales, o X sea vacía, es
    # decir, una vez hallamos comprobado todas las variables predictoras, y todas hayan sido añadidas a S.
    while(True):
        print("S: ")
        print(S)
        # Por cada variable predictora que nos queda por añadir a S
        for i in X:
            sCopy = S.copy()
            # Siempre se evalua con lo que teníamos en S y la nueva variable predictora (solo una nueva!!!)
            sCopy.append(i)
            # Obtenemos train_atts con las variables predictoras que tenemos en S y añadimos la que queremos evaluar
            # en esta iteración (i)
            newAtts = attsConAtributos(atts,sCopy)
            # Evaluamos haciendo una validación cruzada
            puntuacionSX = cross_val_score(tree.DecisionTreeClassifier(random_state=seed), newAtts , label, cv=3, scoring="accuracy")
            puntuacionSX = puntuacionSX.mean()
            print("Puntuacion",puntuacionSX,"incluyendo ",i)
            # Si la puntuacion obtenida ha sido mejor que la que ya teniamos, la almacenamos y nos guardamos la 
            # variable predictora que ha provocado dicha puntuacion
            if puntuacionSX>nuevaPuntuacion:
                nuevaPuntuacion = puntuacionSX
                Xmejor = i
        # Tras comprobar las variables predictoras contenidas en X, miramos si la puntuacion que teníamos del
        # sistema es peor que la nueva puntuacion, es decir, es peor que si añadiesemos la nueva variable predictora
        # , Xmejor
        # Si se da el caso, la añadimos a S y la quitamos de X
        if puntuacionS<nuevaPuntuacion:
            print("El mejor ha sido: " , Xmejor , "con puntuacion: ", nuevaPuntuacion)
            S.append(Xmejor)
            X.remove(Xmejor)
        # En caso de que la puntuacion de la iteracion anterior haya sido igual que la puntuacion del sistema o se
        # hayan añadido todas las variables de X en S, procedemos a salir del bucle.
        # Condición de salida:
        if nuevaPuntuacion == puntuacionS or len(X)==0:
            print("No se ha presenciado ninguna variable que mejore el conjunto anterior")
            break
        # Si no se ha dado el caso, volvemos a evaluar el sistema con lo contenido hasta ahora en S y 
        # actualizamos puntuacionS y nuevaPuntuacion, con el fin de tratar en otra iteracion de modificar 
        # nuevaPuntuacion (puntuacionSX) con un Score mayor que el de puntuacionS
        puntuacionS = cross_val_score(tree.DecisionTreeClassifier(random_state=seed), attsConAtributos(atts,S), label, cv=3, scoring="accuracy")
        puntuacionS = puntuacionS.mean()
        nuevaPuntuacion = puntuacionS
    # Se devuelve una lista con las variables que han producido una mejora del Score tras ser añadidas al sistema
    return S

Para comprobar el funcionamiento de nuestro algoritmo, vamos a hacer una prueba con el dataset Pima y Wisconsin; y lo compararemos con los atributos seleccionados por el filter de selección de rankings

In [64]:
var = forward(train_atts[1],train_label[1],seed)
print("Variables con mejor Accuracy - Nuestro algoritmo: ")
print(var)

S: 
[]
Puntuacion 0.680722127039 incluyendo  preg
Puntuacion 0.687305666603 incluyendo  plas
Puntuacion 0.63681071134 incluyendo  pres
Puntuacion 0.626943333968 incluyendo  skin
Puntuacion 0.633606193286 incluyendo  insu
Puntuacion 0.613950758297 incluyendo  mass
Puntuacion 0.573323180405 incluyendo  pedi
Puntuacion 0.669268354591 incluyendo  age
El mejor ha sido:  plas con puntuacion:  0.687305666603
S: 
['plas']
Puntuacion 0.682340250016 incluyendo  preg
Puntuacion 0.66777714322 incluyendo  pres
Puntuacion 0.695364553588 incluyendo  skin
Puntuacion 0.643378386953 incluyendo  insu
Puntuacion 0.692334538994 incluyendo  mass
Puntuacion 0.675899486008 incluyendo  pedi
Puntuacion 0.670965797322 incluyendo  age
El mejor ha sido:  skin con puntuacion:  0.695364553588
S: 
['plas', 'skin']
Puntuacion 0.659607208579 incluyendo  preg
Puntuacion 0.711815470525 incluyendo  pres
Puntuacion 0.671076844977 incluyendo  insu
Puntuacion 0.67593121391 incluyendo  mass
Puntuacion 0.680817310743 incluyend

Resultado obtenido con filter:
- (0.1288987268794517, 'plas'),
- (0.094889792970892284, 'mass'),
- (0.067196685611855322, 'age'),
- (0.064457008631584101, 'preg'),
- (0.040198824529407595, 'pres'),
- (0.035925099264167093, 'insu'),
- (0.014890438929848759, 'skin'),
- (0.0, 'pedi')]

Podemos ver, que más o menos las variables obtenidas son similares, siendo la más importante plas. Aunque hay que tener en cuenta, que nuestro algoritmo hace una combinación de todas las variables y es por ello que obtenemos que hay variables más importantes junto con otras, y sin embargo filter las analiza de manera individual.

In [65]:
var = forward(train_atts[2],train_label[2],seed)
print("Variables con mejor Accuracy - Nuestro algoritmo: ")
print(var)

S: 
[]
Puntuacion 0.858672110095 incluyendo  clumpThickness
Puntuacion 0.932005060089 incluyendo  cellSize
Puntuacion 0.930222528894 incluyendo  CellShape
Puntuacion 0.844421444042 incluyendo  marginalAdhesion
Puntuacion 0.880196653441 incluyendo  epithelialSize
Puntuacion 0.889128476415 incluyendo  bareNuclei
Puntuacion 0.910557184751 incluyendo  blandChromatin
Puntuacion 0.896287351695 incluyendo  normalNucleoli
Puntuacion 0.799599409656 incluyendo  mitoses
El mejor ha sido:  cellSize con puntuacion:  0.932005060089
S: 
['cellSize']
Puntuacion 0.926657466506 incluyendo  clumpThickness
Puntuacion 0.928459164702 incluyendo  CellShape
Puntuacion 0.928459164702 incluyendo  marginalAdhesion
Puntuacion 0.944540279455 incluyendo  epithelialSize
Puntuacion 0.948114925345 incluyendo  bareNuclei
Puntuacion 0.923063653614 incluyendo  blandChromatin
Puntuacion 0.942757748261 incluyendo  normalNucleoli
Puntuacion 0.930203361892 incluyendo  mitoses
El mejor ha sido:  bareNuclei con puntuacion:  0.

Resultado obtenido con filter:
- (0.48603098768879649, 'cellSize'),
- (0.46993766833372019, 'CellShape'),
- (0.41167880380619892, 'bareNuclei'),
- (0.37911994505258395, 'blandChromatin'),
- (0.35302212105383335, 'epithelialSize'),
- (0.31501779705255006, 'clumpThickness'),
- (0.31169836388044914, 'normalNucleoli'),
- (0.30767797478207859, 'marginalAdhesion'),
- (0.14952150918842766, 'mitoses')]

Como podemos ver, la explicación es similar, se eligen variables muy similares, han elegido de igual manera las variables más importantes "cellSize","Cellshape" y las menos importantes como "mitoses" las desprecia.